<a href="https://colab.research.google.com/github/abldvd/CI-Proyects/blob/main/MusicClassifier_SoundConv1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install pydub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = 'drive/MyDrive/Colab Notebooks/datasets/music_files/test'
!ls drive/MyDrive/'Colab Notebooks'/datasets/music_files/test

Classical  Rock  Synthwave


In [ ]:
import os
import math
import librosa
import numpy as np
import random as rd
from pydub import AudioSegment 
from pydub.utils import make_chunks
from scipy.io import wavfile
from tempfile import mktemp
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Dense, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def getNRandomChunksOfMusic(mp3_path, chunk_len, n_chunks):
  mp3_audio = AudioSegment.from_file(mp3_path, format="mp3")  # read mp3
  if (chunk_len > mp3_audio.duration_seconds):
      raise Exception('Fixed lenght greater than file lenght')
  wname = mktemp('.wav')  # use temporary file
  mp3_audio.export(wname, format="wav")
  wav_audio = AudioSegment.from_file_using_temporary_files(wname, format="wav").set_channels(1).set_frame_rate(48000)
  os.remove(wname)      
  chunks = make_chunks(wav_audio, chunk_len*1000) # split and convert from pydub to np 
  chunks = [chunk for chunk in chunks if chunk.duration_seconds == chunk_len]
  chunks = rd.choices(chunks, k=n_chunks)    # dont ask how I reshaped it... I just copied it from somewhere else
  return [np.array(chunk.get_array_of_samples(), dtype=np.float32).reshape(
      (-1, chunk.channels)) / (1 << (8 * chunk.sample_width - 1)) 
      for chunk in chunks]

In [ ]:
def loadSoundWindowData(path, subset=None, valid_perc=None, seed=1234, file_limit=100, samples_per_file=5, sample_len=2):
  # Loads the sliced mp3s into a numpy array
  chunk_shape = getNRandomChunksOfMusic(f'{path}/{os.listdir(path)[0]}/{os.listdir(f"{path}/{os.listdir(path)[0]}")[0]}', sample_len, 1)[0].shape
  if subset == "validation":
    num_files = int(sum([len(os.listdir(f'{path}/{class_folder}')[:file_limit]) for class_folder in os.listdir(path)])*valid_perc)
  elif subset == "training":
    num_files = int(sum([len(os.listdir(f'{path}/{class_folder}')[:file_limit]) for class_folder in os.listdir(path)])*(1 - valid_perc))
  else:
    num_files = int(sum([len(os.listdir(f'{path}/{class_folder}')[:file_limit]) for class_folder in os.listdir(path)]))
  num_classes = len(os.listdir(path))
              # init array
  X = np.zeros((num_files*samples_per_file,)+chunk_shape, dtype='float32')
  y = np.zeros((num_files*samples_per_file,), dtype=str)

  rd.seed(seed)
  last_pos = 0
  for class_folder in os.listdir(path): # Iterating over the classes
    file_list = os.listdir(f'{path}/{class_folder}')
    rd.shuffle(file_list)
    if file_limit:       # Appliying memory limits and randomizing
      file_list = file_list[:file_limit]
      rd.shuffle(file_list)

    if subset == 'training' and valid_perc: # Splitting into validation and training
      file_list = file_list[int(np.floor(len(file_list)*valid_perc)):]
    elif subset == 'validation' and valid_perc:
      file_list = file_list[:int(np.floor(len(file_list)*valid_perc))]
    
    for i, file_name in enumerate(file_list): # Iterating over files
      samples = getNRandomChunksOfMusic(f'{path}/{class_folder}/{file_name}', sample_len, samples_per_file)
      for j, chunk in enumerate(samples):        
        X[i+j+last_pos,] = chunk
        y[i+j+last_pos] = class_folder
        
      last_pos += j
    last_pos += i+1

  lb = LabelEncoder() # Transforming y to categorical
  y = keras.utils.to_categorical(lb.fit_transform(y), num_classes=num_classes)
  return X, y      

In [ ]:
# DATA -------------------------------------------------------------------------
num_classes = len(os.listdir(PATH))
shape = getNRandomChunksOfMusic(f'{PATH}/{os.listdir(PATH)[0]}/{os.listdir(f"{PATH}/{os.listdir(PATH)[0]}")[0]}', 2, 1)[0].shape
training = tf.data.Dataset.from_tensor_slices(loadSoundWindowData(PATH, subset='training', valid_perc=0.2))
validation = tf.data.Dataset.from_tensor_slices(loadSoundWindowData(PATH, subset='validation', valid_perc=0.2))

In [ ]:
# MODEL ------------------------------------------------------------------------
model = keras.Sequential()
model.add(BatchNormalization(input_shape=shape))
model.add(Conv1D(32, 3, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(2))
model.add(Conv1D(32, 3, activation='sigmoid'))

model.add(Conv1D(64, 3, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='sigmoid'))

model.add(keras.layers.Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
   

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

In [ ]:
# TRAINING ---------------------------------------------------------------------

epochs = 200

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=25, restore_best_weights=True)

h = model.fit(
        training.batch(30),
        epochs=epochs, 
        validation_data=validation.batch(30),
        #callbacks = [es]
)

Epoch 1/200
40/40 [==============================] - 328s 8s/step - loss: 831.1799 - accuracy: 0.5250 - val_loss: 65.9873 - val_accuracy: 0.3333
Epoch 2/200
40/40 [==============================] - 316s 8s/step - loss: 39.4353 - accuracy: 0.4250 - val_loss: 148.6696 - val_accuracy: 0.3333
Epoch 3/200
40/40 [==============================] - 322s 8s/step - loss: 22.4234 - accuracy: 0.3333 - val_loss: 40.8474 - val_accuracy: 0.3333
Epoch 4/200
40/40 [==============================] - 322s 8s/step - loss: 4.9257 - accuracy: 0.3333 - val_loss: 30.4308 - val_accuracy: 0.3333
Epoch 5/200
40/40 [==============================] - 328s 8s/step - loss: 3.4435 - accuracy: 0.3333 - val_loss: 24.2251 - val_accuracy: 0.3333
Epoch 6/200
40/40 [==============================] - 331s 8s/step - loss: 2.5958 - accuracy: 0.3333 - val_loss: 20.0146 - val_accuracy: 0.3333
Epoch 7/200
16/40 [===========>..................] - ETA: 3:07 - loss: 3.2330 - accuracy: 0.0000e+00

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'])
plt.plot(h.history['loss'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation','loss'], loc='upper right')
plt.show()